In [2]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
  
import os  
# print(os.listdir("../input"))  

In [3]:
data = pd.read_csv("database.csv")  
data.head()  

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [4]:
data.columns  


Index(['Date', 'Time', 'Latitude', 'Longitude', 'Type', 'Depth', 'Depth Error',
       'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status'],
      dtype='object')

In [5]:
data = data[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]  
data.head()  

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,01/02/1965,13:44:18,19.246,145.616,131.6,6.0
1,01/04/1965,11:29:49,1.863,127.352,80.0,5.8
2,01/05/1965,18:05:58,-20.579,-173.972,20.0,6.2
3,01/08/1965,18:49:43,-59.076,-23.557,15.0,5.8
4,01/09/1965,13:32:50,11.938,126.427,15.0,5.8


In [6]:
import datetime  
import time  
  
timestamp = []  
for d, t in zip(data['Date'], data['Time']):  
    try:  
        ts = datetime.datetime.strptime(d+' '+t, '%m/%d/%Y %H:%M:%S')  
        timestamp.append(time.mktime(ts.timetuple()))  
    except ValueError:  
        # print('ValueError')  
        timestamp.append('ValueError')  

In [7]:
timeStamp = pd.Series(timestamp)  
data['Timestamp'] = timeStamp.values  

In [8]:
final_data = data.drop(['Date', 'Time'], axis=1)  
final_data = final_data[final_data.Timestamp != 'ValueError']  
final_data.head()  

,Latitude,Longitude,Depth,Magnitude,Timestamp
0,19.246,145.616,131.6,6.0,-157650342.0
1,1.863,127.352,80.0,5.8,-157485611.0
2,-20.579,-173.972,20.0,6.2,-157375442.0
3,-59.076,-23.557,15.0,5.8,-157113617.0
4,11.938,126.427,15.0,5.8,-157046230.0


In [9]:
# from mpl_toolkits.basemap import Basemap  
  
# m = Basemap(projection='mill',llcrnrlat=-80,urcrnrlat=80, llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')  
  
# longitudes = data["Longitude"].tolist()  
# latitudes = data["Latitude"].tolist()  
# #m = Basemap(width=12000000,height=9000000,projection='lcc',  
#             #resolution=None,lat_1=80.,lat_2=55,lat_0=80,lon_0=-107.)  
# x,y = m(longitudes,latitudes)  

In [10]:
X = final_data[['Timestamp', 'Latitude', 'Longitude']]  
y = final_data[['Magnitude', 'Depth']]  

In [11]:
from sklearn.model_selection import train_test_split
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)  

(18727, 3) (4682, 3) (18727, 2) (4682, 3)


In [16]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(random_state=42)  
reg.fit(X_train, y_train)  
reg.predict(X_test)  

array([[  5.862,  40.987],
       [  5.799,  33.306],
       [  6.077,  40.516],
       ...,
       [  6.326,  22.906],
       [  5.856, 593.098],
       [  5.833,  37.739]])

In [13]:
from sklearn.model_selection import GridSearchCV  
  
parameters = {'n_estimators':[10, 20, 50, 100, 200, 500]}  
  
grid_obj = GridSearchCV(reg, parameters)  
grid_fit = grid_obj.fit(X_train, y_train)  
best_fit = grid_fit.best_estimator_  
best_fit.predict(X_test)  


array([[  5.8844 ,  43.3644 ],
       [  5.8122 ,  31.48516],
       [  6.0146 ,  39.2084 ],
       ...,
       [  6.325  ,  22.3666 ],
       [  5.911  , 592.0704 ],
       [  5.7798 ,  39.3888 ]])

In [14]:
best_fit.score(X_test, y_test)  


0.39698694148325175

In [17]:
from keras.models import Sequential  
from keras.layers import Dense  
  
def create_model(neurons, activation, optimizer, loss):  
    model = Sequential()  
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))  
    model.add(Dense(neurons, activation=activation))  
    model.add(Dense(2, activation='softmax'))  
      
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  
      
    return model  

In [26]:
# import keras classifier
# %pip install keras
from scikeras.wrappers import KerasClassifier  

model = KerasClassifier(build_fn=create_model, verbose=0,activation='sigmoid', optimizer='SGD', loss='squared_hinge',neurons=16,batch_size=10,epochs=10,validation_split=0.2)  
  
# neurons = [16, 64, 128, 256]  
neurons = [16]  
# batch_size = [10, 20, 50, 100]  
batch_size = [10]  
epochs = [10]  
# activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'exponential']  
activation = ['sigmoid', 'relu']  
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']  
optimizer = ['SGD', 'Adadelta']  
loss = ['squared_hinge']  
  
param_grid = dict(neurons=neurons, batch_size=batch_size, epochs=epochs, activation=activation, optimizer=optimizer, loss=loss)  

In [30]:
import numpy
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)  

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))  
means = grid_result.cv_results_['mean_test_score']  
stds = grid_result.cv_results_['std_test_score']  
params = grid_result.cv_results_['params']  
for mean, stdev, param in zip(means, stds, params):  
    print("%f (%f) with: %r" % (mean, stdev, param))  

NameError: name 'means' is not defined

In [31]:
model = Sequential()  
model.add(Dense(16, activation='relu', input_shape=(3,)))  
model.add(Dense(16, activation='relu'))  
model.add(Dense(2, activation='softmax'))  
  
model.compile(optimizer='SGD', loss='squared_hinge', metrics=['accuracy'])  

In [36]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

model.fit(X_train, y_train, batch_size=10, epochs=10, validation_split=0.2)

model.evaluate(X_test, y_test)

model.predict(X_test)



Epoch 1/10
1499/1499 [==============================] - 1s 433us/step - loss: 0.5036 - accuracy: 0.9816 - val_loss: 0.5046 - val_accuracy: 0.9789
Epoch 2/10
1499/1499 [==============================] - 1s 415us/step - loss: 0.5036 - accuracy: 0.9816 - val_loss: 0.5046 - val_accuracy: 0.9789
Epoch 3/10
1499/1499 [==============================] - 1s 431us/step - loss: 0.5036 - accuracy: 0.9816 - val_loss: 0.5046 - val_accuracy: 0.9789
Epoch 4/10
1499/1499 [==============================] - 1s 420us/step - loss: 0.5036 - accuracy: 0.9816 - val_loss: 0.5046 - val_accuracy: 0.9789
Epoch 5/10
1499/1499 [==============================] - 1s 414us/step - loss: 0.5036 - accuracy: 0.9816 - val_loss: 0.5046 - val_accuracy: 0.9789
Epoch 6/10
1499/1499 [==============================] - 1s 414us/step - loss: 0.5036 - accuracy: 0.9816 - val_loss: 0.5046 - val_accuracy: 0.9789
Epoch 7/10
1499/1499 [==============================] - 1s 421us/step - loss: 0.5036 - accuracy: 0.9816 - val_loss: 0.5046 -

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [35]:
[test_loss, test_acc] = model.evaluate(X_test, y_test)  
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))  

147/147 [==============================] - 0s 380us/step - loss: 0.5038 - accuracy: 0.9814
Evaluation result on Test Data : Loss = 0.5038455724716187, accuracy = 0.9814181923866272


In [37]:
# RNN model

from keras.layers import SimpleRNN, Embedding
from keras.layers import LSTM
from keras.models import Sequential  
from keras.layers import Dense  

model = Sequential()  
model.add(Embedding(10001, output_dim=16, input_length=3))  # Add an Embedding layer with input_dim=10001
model.add(SimpleRNN(16))  # Add a SimpleRNN layer
model.add(Dense(2, activation='softmax'))  # Add a Dense layer

model.compile(optimizer='SGD', loss='squared_hinge', metrics=['accuracy'])  


X_train = X_train.astype('float32')
y_train = y_train.astype('float32')



X_train['Timestamp'] = X_train['Timestamp'] / 1000000000
X_test['Timestamp'] = X_test['Timestamp'] / 1000000000

X_train['Latitude'] = X_train['Latitude'] + 100
X_test['Latitude'] = X_test['Latitude'] + 100

X_train['Longitude'] = X_train['Longitude'] + 200
X_test['Longitude'] = X_test['Longitude'] + 200



model.fit(X_train, y_train, batch_size=10, epochs=10, validation_split=0.2)


Epoch 1/10
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0036 - accuracy: 0.3672 - val_loss: 0.0048 - val_accuracy: 0.4130
Epoch 2/10
1499/1499 [==============================] - 2s 1ms/step - loss: 0.0036 - accuracy: 0.4590 - val_loss: 0.0048 - val_accuracy: 0.4754
Epoch 3/10
1499/1499 [==============================] - 2s 1ms/step - loss: 0.0036 - accuracy: 0.5004 - val_loss: 0.0048 - val_accuracy: 0.5411
Epoch 4/10
1499/1499 [==============================] - 2s 1ms/step - loss: 0.0036 - accuracy: 0.5990 - val_loss: 0.0048 - val_accuracy: 0.6356
Epoch 5/10
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0036 - accuracy: 0.6527 - val_loss: 0.0048 - val_accuracy: 0.6975
Epoch 6/10
1499/1499 [==============================] - 2s 1ms/step - loss: 0.0036 - accuracy: 0.7312 - val_loss: 0.0048 - val_accuracy: 0.7640
Epoch 7/10
1499/1499 [==============================] - 2s 1ms/step - loss: 0.0036 - accuracy: 0.7865 - val_loss: 0.0048 - val_accuracy:

In [38]:
X_test_indices = X_test.astype(int)
X_test_indices = np.clip(X_test_indices, 0, 10000)

[test_loss, test_acc] = model.evaluate(X_test_indices, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))


147/147 [==============================] - 0s 857us/step - loss: 0.0039 - accuracy: 0.9082
Evaluation result on Test Data : Loss = 0.003918507602065802, accuracy = 0.9081588983535767


In [43]:
def getPredictions(model, X_test):
    # X_test_indices = X_test.astype(int)
    # X_test_indices = np.clip(X_test_indices, 0, 10000)
    pred = model.predict(X_test)
    return pred

print(X_test)
predictions = getPredictions(model, X_test)
print()

print(predictions[:5])
print(y_test[:5])


          Timestamp    Latitude   Longitude
13121  8.386518e-10  124.601997  322.281982
17921  1.151164e-09   99.610001  323.195007
3419   1.656469e-10   76.977997   24.888000
20803  1.306189e-09  139.709000  343.247009
15057  9.720515e-10   95.376999   93.956001
...             ...         ...         ...
10828  7.019929e-10   50.127998   86.255997
22412  1.413073e-09  157.281998  166.733307
18333  1.174765e-09   79.246002  369.354004
11266  7.254732e-10   93.913002  313.049988
9152   5.886828e-10   84.136002   27.932999

[4682 rows x 3 columns]
147/147 [==============================] - 0s 1ms/step

[[0.48406318 0.51593685]
 [0.4933851  0.5066149 ]
 [0.48855925 0.51144075]
 [0.46720293 0.5327971 ]
 [0.5018415  0.4981585 ]]
       Magnitude      Depth
13121        5.5  74.300003
17921        6.3  26.000000
3419         5.6  33.000000
20803        5.8  16.000000
15057        5.5  10.000000
